In [10]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from os import getenv
from openai import OpenAI
import pathlib
import textwrap
import google.generativeai as genai
import time
import ast
load_dotenv("../.env",override=True)
GOOGLE_API_KEY = getenv("GEMINI_API_KEY")
OPENAI_API_KEY = getenv("OPENAI_API_KEY")



In [2]:
sample_df = pd.read_csv("sample_df_with_onet_similarity.csv")

In [24]:

results = []
for i,row in sample_df.iterrows():
    name = row["organization name"]
    for x in range(1,3):
        onet_col = f"Job{x}_ONET"
        job = row[f"Job{x}"]
        example = row[f"Example{x}"]
        onet_vals = ast.literal_eval(row[onet_col])
        for onet in onet_vals:
            if float(onet[2]) > .47 and float(onet[3])>.25:
                results.append([name, example, job]+onet)


onet_df = pd.DataFrame(results, columns=["organization_name","example","job","onet_title","onet_task","task_similarity", "job_title_similarity"])



In [25]:
len(onet_df.organization_name.unique())

136

In [31]:
onet_df.onet_title.value_counts().reset_index().values

array([['Market Research Analysts and Marketing Specialists', 42],
       ['Financial and Investment Analysts', 26],
       ['Customer Service Representatives', 18],
       ['Project Management Specialists', 16],
       ['Web and Digital Interface Designers', 14],
       ['Sales Representatives of Services, Except Advertising, Insurance, Financial Services, and Travel',
        12],
       ['Business Intelligence Analysts', 12],
       ['Legal Secretaries and Administrative Assistants', 11],
       ['Lawyers', 11],
       ['Interior Designers', 10],
       ['Marketing Managers', 9],
       ['Fraud Examiners, Investigators and Analysts', 9],
       ['Video Game Designers', 9],
       ['Credit Analysts', 9],
       ['Accountants and Auditors', 8],
       ['Investment Fund Managers', 8],
       ['Web Developers', 7],
       ['Educational, Guidance, and Career Counselors and Advisors', 7],
       ['Online Merchants', 7],
       ['Medical Assistants', 7],
       ['Search Marketing Strategis

In [41]:
onet_weights = onet_df.groupby("organization_name")["onet_title"].count().apply(lambda x: 1/x).reset_index().rename({"onet_title":"onet_weight"},axis=1)



In [43]:
onet_df = onet_df.merge(onet_weights, on="organization_name")

In [51]:
onet_df.groupby("onet_title")["onet_weight"].sum().round(2).sort_values(ascending=False).reset_index().values

array([['Market Research Analysts and Marketing Specialists', 8.5],
       ['Customer Service Representatives', 4.97],
       ['Financial and Investment Analysts', 4.92],
       ['Investment Fund Managers', 3.5],
       ['Business Intelligence Analysts', 3.33],
       ['Lawyers', 3.25],
       ['Search Marketing Strategists', 2.83],
       ['Video Game Designers', 2.8],
       ['Project Management Specialists', 2.67],
       ['Legal Secretaries and Administrative Assistants', 2.67],
       ['Web and Digital Interface Designers', 2.6],
       ['Online Merchants', 2.5],
       ['Sales Representatives of Services, Except Advertising, Insurance, Financial Services, and Travel',
        2.48],
       ['Data Scientists', 2.33],
       ['Human Resources Specialists', 2.0],
       ['Fundraising Managers', 2.0],
       ['Fraud Examiners, Investigators and Analysts', 2.0],
       ['Photographers', 2.0],
       ['Marketing Managers', 2.0],
       ['Accountants and Auditors', 1.9],
       ['Traini

In [52]:
onet_df.to_csv("onet_df.csv")

In [53]:
onet_df.values

array([['BLOVO Animal Health',
        "A veterinarian uses a platform to analyze a pet's medical history and current health data to predict potential future health issues. This allows them to proactively recommend preventive measures and early interventions, such as diet adjustments or specific medications.",
        'A veterinarian analyzes medical and health data to predict and diagnose health issues.',
        ..., 0.5644843187073701, 0.513404694847263, 0.16666666666666666],
       ['BLOVO Animal Health',
        "A veterinarian uses a platform to analyze a pet's medical history and current health data to predict potential future health issues. This allows them to proactively recommend preventive measures and early interventions, such as diet adjustments or specific medications.",
        'A veterinarian analyzes medical and health data to predict and diagnose health issues.',
        ..., 0.560867075190992, 0.513404694847263, 0.16666666666666666],
       ['BLOVO Animal Health',
  